<a href="https://colab.research.google.com/github/DACSS-690C/Example/blob/main/Spatial_Data_Intro_Projection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

<center><img src="https://github.com/DACSS-CSSmeths/guidelines/blob/main/pics/small_logo_ccs_meths.jpg?raw=true" width="900"></center>

# Introduction

I have downloaded a couple of  maps from the [MassGis](https://www.mass.gov/orgs/massgis-bureau-of-geographic-information) website as shapefiles:

* [Counties](https://www.mass.gov/info-details/massgis-data-counties)
* [Roads](https://www.mass.gov/info-details/massgis-data-massgis-massdot-roads)

The shapefiles are on GitHub. Let me read them:


In [ ]:
import geopandas as gpd

# the links
mainLocation='https://github.com/DACSS-CSSmeths/Spatial-intro/raw/refs/heads/main/maps/'
linkCountiesMA=mainLocation+'MA_maps/counties/COUNTIESSURVEY_POLYM.shp'
linkRoadsMA=mainLocation+'MA_maps/roads/EOTMAJROADS_RTE_MAJOR.shp'

#reading as geoDataFrame

counties=gpd.read_file(linkCountiesMA)
roads=gpd.read_file(linkRoadsMA)

Take look at both of them:

In [ ]:
base=counties.plot(color='lightgrey', edgecolor='black')
roads.plot(ax=base,color='red')

# The map projection

The earth is not flat. But, the plot above is a flat representation of MA. That representation is possible because an expert cartographer created an formula to represent a spherical-like surface into a plane. There are flat maps that are technically speaking nonprojected (geographic).


# The Coordinate Reference System

You have projection, then you may find a location. That location has a particular coordinate, that is the CRS. Any map has a CRS, but that CRS can be a geographic (non projected) or projected CRS. For visualization purposes, both can be used, but several important spatial operations are only valid on map with a projected CRS.

You can find these technical information here:

In [ ]:
counties.crs

In [ ]:
roads.crs

A simpler way to know if the CRS is projected:

In [ ]:
roads.crs.is_projected

We can create a map of MA borders by dissolving the counties:

In [ ]:
borders=counties.dissolve()
borders.plot(color='lightgrey', edgecolor='black')

And, as you see:

In [ ]:
borders.crs

Can we compute the centroid?

In [ ]:
# easy peasy
borders.centroid

But, what happens with geographic CRS?
Let me use the USA map I have:

In [ ]:


link_USA=mainLocation+'USA_boundaries/united_states_of_america_United_States_of_America_Country_Boundary.shp'

#reading as geoDataFrame

borderUSA=gpd.read_file(link_USA)

# get a warning!
borderUSA.centroid

Then, let me show you next how to turn modify this,

# Formatting a GeoDataframe from coordinates

It is very common to have longitude and latitude information in simple *dataframes*.

In [ ]:
import pandas as pd
linkAirUSA=mainLocation+'MA_maps/airports.csv'
airportsUSA=pd.read_csv(linkAirUSA)
airportsUSA.head()

Those coordinates can help create a *Geodataframe*.


## From DataFrame to GeoDataFrame


First, create the GeoDataFrame, notice the **CRS**:



In [ ]:
airportsUSA_gdf=gpd.GeoDataFrame(data=airportsUSA,
                 geometry=gpd.points_from_xy(airportsUSA.LONGITUDE,
                                             airportsUSA.LATITUDE),
                 crs=4326)# the coordinates were  unprojected

We used 4326 because long/lat are created using that CRS. So now we have:

In [ ]:
type(airportsUSA_gdf),type(airportsUSA)

## Reprojecting CRS

We know:

In [ ]:
airportsUSA_gdf.crs.is_projected

In [ ]:
airportsUSA_gdf.crs==borders.crs

Let me keep the MA airports:

In [ ]:
airportsMA_gdf=airportsUSA_gdf[airportsUSA_gdf.STATE=="MA"]
airportsMA_gdf

What if I do not change the CRS?

In [ ]:
base=counties.plot(color='lightgrey', edgecolor='black')

airportsMA_gdf.plot(ax=base,color='red')

Then, this is a critical step, **REPROJECTING**:

In [ ]:
airportsMA_gdf_prjd=airportsMA_gdf.to_crs(counties.crs)

Now:

In [ ]:
base=counties.plot(color='lightgrey', edgecolor='black')

airportsMA_gdf_prjd.plot(ax=base,color='red')

# Saving your work

Since we usually have maps that are closely related, you should consider the **geopackage**:

In [ ]:
counties.to_file('MA_maps.gpkg',layer='counties_poly')
roads.to_file('MA_maps.gpkg',layer='roads_line')
borders.to_file('MA_maps.gpkg',layer='borders_line')
airportsMA_gdf_prjd.to_file('MA_maps.gpkg',layer='airports_point')

# How to do it in R?

In [ ]:
%load_ext rpy2.ipython

In [ ]:
%%R
# system("apt-get -y update")
# system("apt-get install -y libudunits2-dev libgdal-dev libgeos-dev libproj-dev")
# install.packages("sf")

(as ‘lib’ is unspecified)







































	‘/tmp/RtmpnNuU6Y/downloaded_packages’



1. Read the data from GitHub:

* Remember to add **/vsicurl/**.

In [ ]:
%%R

library(sf)

# the links
mainLocation='https://github.com/DACSS-CSSmeths/Spatial-intro/raw/refs/heads/main/maps/'

linkCountiesMA=paste0('/vsicurl/',mainLocation,'MA_maps/counties/COUNTIESSURVEY_POLYM.shp')
linkRoadsMA=paste0('/vsicurl/',mainLocation,'MA_maps/roads/EOTMAJROADS_RTE_MAJOR.shp')
link_USA=paste0('/vsicurl/',mainLocation,'USA_boundaries/united_states_of_america_United_States_of_America_Country_Boundary.shp')

#reading as geoDataFrame

counties=read_sf(linkCountiesMA)
roads=read_sf(linkRoadsMA)
borderUSA=read_sf(link_USA)

2. Dissolving several polygons into one

* Using **st_union**

In [ ]:
%%R

borders=st_union(counties, by_feature = FALSE)

# see dissolved polygons

library(ggplot2)

base=ggplot(borders) + geom_sf()

base


3. Identify the CRS

* using **st_crs**

In [ ]:
%%R
st_crs(borders)$epsg

In [ ]:
%%R
st_crs(borderUSA)$epsg


4. Check if CRS is projected

* The function **st_is_longlat()** ask if the CRS is geographic (not projected)

In [ ]:
%%R
st_is_longlat(borderUSA)

In [ ]:
%%R
st_is_longlat(borders)

5. Request centroid

* you get an spatial object with **st_centroid()**

In [ ]:
%%R

theCentroid_MA=st_centroid(borders)

#see it
base + geom_sf(data = theCentroid_MA,
               color = "red", size = 3)

In [ ]:
%%R

theCentroid_USA=st_centroid(borderUSA)

### see it
ggplot(borderUSA) + geom_sf() + geom_sf(data = theCentroid_USA,
               color = "red", size = 3)

5. From DF to GeoDF

In [ ]:
%%R


linkAirUSA=paste0(mainLocation,'MA_maps/airports.csv')

# the DF
airportsUSA=read.csv(linkAirUSA)

# create the GeoDF
airportsUSA_gdf=st_as_sf(airportsUSA,
                         coords = c("LONGITUDE","LATITUDE"),
                         crs=4326) # geographic CRS for lon/lat

# verifying
st_crs(airportsUSA_gdf)$epsg

6. Reprojecting the CRS

We keep MA airports, and we reproject that submap:

In [ ]:
%%R

airportsMA_gdf=airportsUSA_gdf[airportsUSA_gdf$STATE%in%c("MA"),]
airportsMA_gdf_prjd=st_transform(airportsMA_gdf, 26986)

airportsMA_gdf_prjd

This works well!

In [ ]:
%%R
base + geom_sf(data = airportsMA_gdf_prjd,
               color = "red", size = 3)

From the previous ColabNotebook you know how to save all these maps into a **geopackage**.